In [5]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import catboost

In [6]:
# Loading Data

train = pd.read_csv('data/train.csv')
songs = pd.read_csv('data/songs.csv')
song_labels = pd.read_csv('data/song_labels.csv')
test = pd.read_csv('data/test.csv')
save_for_later = pd.read_csv('data/save_for_later.csv')
dummy_submission = pd.read_csv('data/dummy_submission.csv')

In [4]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(train, train_size = 0.8)

In [ ]:
customer_id_list = train['customer_id'].unique()

In [ ]:
song_id_list = train['song_id'].unique()

In [ ]:
learning_rate = 0.01
iters = 10
dim = 10
# Initialization

customer_weights = {}
for customer in customer_id_list:
    customer_weights[customer] = np.random.uniform(0, 10e-10, dim)
    
song_weights = {}
for song in song_id_list:
    song_weights[song] = np.random.uniform(0, 10e-10, dim)

In [ ]:
# Training
import time
start = time.time()
for i in range(iters):
    for k in range(len(train.index.to_numpy())):
        customer_weight = customer_weights[train['customer_id'].iloc[k]]
        song_weight = song_weights[train['song_id'].iloc[k]]
        y = train['score'].iloc[k]
        
        temp = y - np.dot(customer_weight, song_weight)
        customer_weight_new = customer_weight + learning_rate*temp*song_weight
        song_weight_new = song_weight + learning_rate*temp*customer_weight
        customer_weights[train['customer_id'].iloc[k]] = customer_weight_new
        song_weights[train['song_id'].iloc[k]] = song_weight_new
    print(i)
            
print(time.time()-start)

In [ ]:
customer_weights['O29219']

In [ ]:
song_weights[3459]

In [ ]:
train

In [ ]:
customer_weights_df = pd.DataFrame(customer_weights)

In [ ]:
customer_weights_df = customer_weights_df.transpose()

In [ ]:
customer_weights_df['customer_id'] = customer_weights_df.index

In [ ]:
customer_weights_df.reset_index(inplace = True)

In [ ]:
customer_weights_df.drop('index', axis = 1, inplace = True)

In [ ]:
customer_weights_df

In [ ]:
train = train.merge(customer_weights_df, on = 'customer_id', how = 'left')

In [ ]:
song_weights_df = pd.DataFrame(song_weights)

In [ ]:
song_weights_df = song_weights_df.transpose()

In [ ]:
song_weights_df['song_id'] = song_weights_df.index

In [ ]:
songs = songs.merge(song_weights_df, on = 'song_id', how = 'left')

In [ ]:
estimates = []
for k in range(len(train.index.to_numpy())):
    customer_weight = customer_weights[train['customer_id'].iloc[k]]
    song_weight = song_weights[train['song_id'].iloc[k]]
    estimate = np.dot(customer_weight, song_weight)
    estimates.append(estimate)

In [ ]:
len(estimates)

In [ ]:
train['estimates'] = estimates

In [ ]:
train

In [ ]:
song_labels_new = song_labels.groupby('label_id').sum().sort_values('count', ascending = False)

In [ ]:
song_labels_new.reset_index(inplace = True)

In [ ]:
principal_song_labels = song_labels_new['label_id'].to_numpy()[:100]

In [ ]:
for i in principal_song_labels:
    label = song_labels[song_labels['label_id'] == i]
    label = label.reset_index().drop(['index', 'label_id'], axis = 1)
    label[f'count{i}'] = label['count']
    label.drop(['count'], axis = 1, inplace = True)
    songs = pd.merge(songs, label, on = 'platform_id', how = 'left')

In [ ]:
for i in principal_song_labels:
    songs[f'count{i}'] = songs[f'count{i}'].fillna(0)

In [ ]:
songs = songs.drop(['platform_id'], axis = 1)

In [ ]:
song_scores = train.merge(train.groupby('song_id').mean(), on = 'song_id', how = 'left')[['song_id', 'score_y']].drop_duplicates('song_id', keep = 'first')

In [ ]:
songs = songs.merge(song_scores, on = 'song_id', how = 'left')

In [ ]:
song_num_ratings = train['song_id'].value_counts().to_frame()

In [ ]:
song_num_ratings['num_ratings'] = song_num_ratings['song_id']

In [ ]:
song_num_ratings['song_id'] = song_num_ratings.index

In [ ]:
songs = songs.merge(song_num_ratings, on = 'song_id', how = 'left')

In [ ]:
songs.drop_duplicates('song_id', keep = 'first', inplace = True)

In [ ]:
songs

In [ ]:
f = pd.merge(train, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')

In [ ]:
train = f

In [ ]:
train

In [ ]:
X_train = pd.merge(train, songs, on = ['song_id'], how = 'left')

In [ ]:
X_train

In [ ]:
Y_train = X_train['score']

In [ ]:
X_train.drop(['score'], axis = 1, inplace = True)

In [ ]:
X_train.drop('song_id', axis = 1, inplace = True)

In [ ]:
X_train['released_year'] = X_train['released_year'].fillna(-999)
X_train['language'] = X_train['language'].fillna('none')
X_train['number_of_comments'] = X_train['number_of_comments'].fillna(-999)

In [ ]:
X_train.drop('customer_id', axis = 1, inplace = True)

In [ ]:
X_train.columns.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, train_size = 0.2)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor()

In [ ]:
eval_dataset = catboost.Pool(data=x_test,
                    label=y_test,
                    cat_features=[11, 13])
model.fit(x_train, y_train, eval_set = eval_dataset, cat_features = [11, 13], use_best_model = True, plot = True)

In [ ]:
test

In [ ]:
test = test.merge(customer_weights_df, on = 'customer_id', how = 'left')

In [ ]:
test

In [ ]:
estimates_test = []
for k in range(len(test.index.to_numpy())):
    customer_weight = customer_weights[test['customer_id'].iloc[k]]
    song_weight = song_weights[test['song_id'].iloc[k]]
    estimate = np.dot(customer_weight, song_weight)
    estimates_test.append(estimate)

In [ ]:
test['estimates'] = estimates_test

In [ ]:
test

In [ ]:
f = pd.merge(test, save_for_later, on=['customer_id','song_id'], how='left', indicator='Exist')

In [ ]:
test = f

In [ ]:
test

In [ ]:
X_test = pd.merge(test, songs, on = ['song_id'], how = 'left')

In [ ]:
X_test

In [ ]:
X_test.drop('song_id', axis = 1, inplace = True)

In [ ]:
X_test['released_year'] = X_test['released_year'].fillna(-999)
X_test['language'] = X_test['language'].fillna('none')
X_test['number_of_comments'] = X_test['number_of_comments'].fillna(-999)

In [ ]:
X_test.drop('customer_id', axis = 1, inplace = True)

In [ ]:
X_test

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
len(y_test_pred)

In [ ]:
y_final = pd.DataFrame(y_test_pred)

In [ ]:
y_final['score'] = y_final[0]

In [ ]:
y_final.drop(0, axis = 1, inplace = True)

In [ ]:
y_final['test_row_id'] = y_final.index

In [ ]:
y_final = y_final[['test_row_id', 'score']]

In [ ]:
y_final

In [ ]:
y_final.to_csv('improvedv3.csv', index = False)

In [7]:
train['customer_id'].nunique()

14053